In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/home/uncookie/Documents/titanic_pipeline/df_after_preprocessing.csv')

In [3]:
def use_lr_to_a_column(df, column):
    from sklearn.linear_model import LinearRegression
    test_data = df[df[column].isnull()]
    df.dropna(inplace=True)
    x_train = df.drop(column,axis=1)
    x_train = x_train.drop('Survived',axis=1)
    y_train = df[column]
    lr = LinearRegression()
    lr.fit(x_train, y_train)
    x_test = test_data[['Pclass','Sex', 'SibSp', 'Parch', 'Fare']]
    y_pred = lr.predict(x_test)
    return y_pred

def fill_nan_with_lr(df):
    df_without_age = df.loc[:, df.columns != 'Age']
    df_without_cabin = df.loc[:, df.columns != 'Cabin']
    a = pd.Series(use_lr_to_a_column(df_without_age, 'Cabin'))
    b = pd.Series(use_lr_to_a_column(df_without_cabin, 'Age'))

    nan_indices = df[df['Cabin'].isna()].index
    for index, replacement_value in zip(nan_indices, a):
        df.at[index, 'Cabin'] = replacement_value

    nan_indices = df[df['Age'].isna()].index
    for index, replacement_value in zip(nan_indices, b):
        df.at[index, 'Age'] = replacement_value

    return df

In [4]:
df = fill_nan_with_lr(df)
df.to_csv("df_after_lr.csv", index=False)

/tmp/ipykernel_45399/1226395653.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/tmp/ipykernel_45399/1226395653.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [5]:
import numpy as np
from sklearn.model_selection import train_test_split

y = df['Survived']
df = df.loc[:, df.columns != 'Survived']

In [6]:
df = (df-df.mean())/df.std()

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)

In [7]:
from tensorflow import keras
from keras import backend as K
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(32, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=4,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="checkpoint_path.keras",
        monitor="val_loss",
        save_best_only=True,
)
]

model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=['accuracy'])

history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    callbacks=callbacks_list,
                    batch_size=16,
                    validation_split=0.2)

2023-12-08 22:37:25.989434: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/20


2023-12-08 22:37:27.155330: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


 1/36 [..............................] - ETA: 17s - loss: 0.6899 - accuracy: 0.3750

23/36 [==================>...........] - ETA: 0s - loss: 0.6171 - accuracy: 0.7038 

...

Epoch 6/20


 1/36 [..............................] - ETA: 0s - loss: 0.4191 - accuracy: 0.8125

15/36 [===========>..................] - ETA: 0s - loss: 0.4514 - accuracy: 0.8167

23/36 [==================>...........] - ETA: 0s - loss: 0.4554 - accuracy: 0.8016

36/36 [==============================] - 0s 7ms/step - loss: 0.4424 - accuracy: 0.8155 - val_loss: 0.4056 - val_accuracy: 0.8182


In [8]:
model.save('lr_nan_model.keras')

with open('acc_of_lr_nan_handling_model.txt', 'w') as file:
    history_dict = history.history
    acc = history_dict['val_accuracy'][-1]
    file.write(f'{acc}')